# Producing test results

In [29]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
import torch.optim as optim

from torch.utils import data

In [42]:
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        y = self.net(x)
        out = self.softmax(y)
        return out

In [43]:
context_size = 10
model = Simple_MLP([40*(2*context_size+1), 256, 128, 71])
state_dict = torch.load("Models/"+"Trial6Epoch30.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [80]:
input_data = np.load("Data/test.npy", allow_pickle=True)

# fake label generation
fake_labels = []
for i in range(len(input_data)):
    shape = input_data[i].shape
    fake_labels.append(np.zeros(shape[0]))

In [81]:
print(shape)

(1402, 40)


In [83]:
fake_labels[1].shape

(363,)

In [84]:
from dataset import MyDataset

In [85]:
input_dataset = MyDataset(input_data, fake_labels, context_size = context_size)
loader_args = dict(shuffle=False, batch_size=1, pin_memory=True)
loader = data.DataLoader(input_dataset, **loader_args)

In [90]:
def test_model(model, loader):
    fields = ["id", "label"]
    rows = []
    with torch.no_grad():
        model.eval()
        input_id = 0
        for batch_idx, (data, target) in enumerate(loader):   
            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            rows.append([input_id, predicted.item()])
            input_id += 1
        return fields, rows

In [92]:
fields, rows = test_model(model, loader)

In [93]:
import csv
with open('Data/out.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)